In [1]:
import numpy as np

In [2]:
import os

clip_folder1 = r'./mean_var_kurt_skew'
data = []
labels=  []

for filename in os.listdir(clip_folder1):
    fname = clip_folder1 + '/' + filename
    labels.append(filename[-5])
    x = np.load(fname)
    data.append(x)

In [3]:
data = np.array(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)

(5587, 512)
(5587,)


### Classifiers

gradient boosting

In [4]:
#loss= deviance

from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse')


In [5]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, data, labels, cv=5)
print(scores)

[0.83542039 0.86762075 0.84870188 0.84959714 0.85944494]


In [6]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.85 (+/- 0.02)


Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, data, labels, cv=5)
print(scores)

[0.82200358 0.85778175 0.85049239 0.84959714 0.83974933]


In [8]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84 (+/- 0.02)


SVM

In [9]:
from sklearn import svm

clf = svm.SVC(kernel = 'poly',verbose = 1,coef0=25)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, data, labels, cv=5)
print(scores)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][0.8059034  0.81484794 0.81468218 0.81557744 0.82273948]


In [10]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.81 (+/- 0.01)


In [11]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
clip_folder1 = r'./mean_var_kurt_skew'
data = []
labels = []

In [13]:
# Load data and labels
for filename in os.listdir(clip_folder1):
    fname = os.path.join(clip_folder1, filename)
    labels.append(filename[-5])
    x = np.load(fname)
    data.append(x)

In [14]:
# Convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [16]:
# Encode the labels to integers
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)

In [18]:
# One-hot encode the integer labels
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded_labels = integer_encoded_labels.reshape(len(integer_encoded_labels), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded_labels)

In [19]:
# Step 2: Train the classifiers and extract last layer representations

# Random Forest
rf_clf = RandomForestClassifier()
rf_clf.fit(data, integer_encoded_labels)
rf_representations = rf_clf.apply(data)  # Extract leaves indices as representations

C:\Users\tarun\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [20]:
# Gradient Boosting
gb_clf = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse')
gb_clf.fit(data, integer_encoded_labels)
gb_representations = gb_clf.apply(data)[:, :, 0]  # Extract decision function values as representations


C:\Users\tarun\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
# SVM
svm_clf = svm.SVC(kernel='poly', verbose=1, coef0=25, decision_function_shape='ovr')
svm_clf.fit(data, integer_encoded_labels)
svm_representations = svm_clf.decision_function(data)  # Extract decision function values as representations


C:\Users\tarun\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]

In [22]:
# Step 3: Concatenate feature vectors
concatenated_features = np.concatenate((rf_representations, gb_representations, svm_representations), axis=1)


In [23]:
# Step 4: Build and train a simple neural network (2 FC layers model)
X_train, X_test, y_train, y_test = train_test_split(concatenated_features, onehot_encoded_labels, test_size=0.2, random_state=42)


In [24]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(concatenated_features.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 output units for 3 classes
])

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/10


140/140 [==============================] - 1s 4ms/step - loss: 22.7812 - accuracy: 0.7592 - val_loss: 15.0516 - val_accuracy: 0.7513
Epoch 2/10
140/140 [==============================] - 0s 1ms/step - loss: 12.5897 - accuracy: 0.7765 - val_loss: 9.3928 - val_accuracy: 0.8050
Epoch 3/10
140/140 [==============================] - 0s 1ms/step - loss: 8.0756 - accuracy: 0.8000 - val_loss: 6.9339 - val_accuracy: 0.7809
Epoch 4/10
140/140 [==============================] - 0s 2ms/step - loss: 6.4732 - accuracy: 0.7966 - val_loss: 9.1435 - val_accuracy: 0.7835
Epoch 5/10
140/140 [==============================] - 0s 1ms/step - loss: 4.5790 - accuracy: 0.7941 - val_loss: 4.7581 - val_accuracy: 0.8184
Epoch 6/10
140/140 [==============================] - 0s 2ms/step - loss: 2.8935 - accuracy: 0.8094 - val_loss: 4.5154 - val_accuracy: 0.7648
Epoch 7/10
140/140 [==============================] - 0s 1ms/step - loss: 2.5112 - accuracy: 0.8053 - val_loss: 3.0604 - val_accuracy: 0.8122


In [26]:
# Step 5: Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


35/35 [==============================] - 0s 558us/step


In [27]:
accuracy = accuracy_score(y_test_classes, y_pred_classes)
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)


In [28]:
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.778175313059034
Confusion Matrix:
[[758 113   9]
 [ 71  78  32]
 [  4  19  34]]
